In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import xlsxwriter
import time
from selenium.common.exceptions import NoSuchElementException


url = "https://www.ratemyprofessors.com"

# sets up the chromedriver for selenium
# PROJECT_ROOT = os.path.abspath(os.path.dirname(__file__))
PROJECT_ROOT=os.path.abspath('')
DRIVER_BIN = os.path.join(PROJECT_ROOT, "chromedriver")
driver = webdriver.Chrome(executable_path=DRIVER_BIN)
driver.get(url)
driver.implicitly_wait(30)
driver.maximize_window()

# sets up an excel workbook/worksheet
workbook = xlsxwriter.Workbook("professorRatings.xlsx")
worksheet = workbook.add_worksheet("Professors")
row = 0
column = 0

# writes column headings to the excel sheet
headings = ["Prof_name", "dept_name", "time", "course", "quality", "difficulty", "reviews", "credit", "attendance",
            "grade", "take_again", "textbook", "online", "helpful", "unhelpful", "tag1", "tag2", "tag3", "tag4", "tag5",
            "tag6", "tag7", "tag8", "tag9", "tag10"]
for item in headings:
    worksheet.write(row, column, item)
    column += 1
row = row + 1
column = 0


# creates the review_ratings method: reads the reviews of each individual professor into an excel file
def review_ratings(id_num):
    global row, column
    print("Entered Method")
    time.sleep(1)
    #  Creates custom professor url based on the 'id' of the list item of each professor
    #prof_url = url + "ShowRatings.jsp?tid=" + str(id_num) + "&showMyProfs=true" # modified
    prof_url = url +id_num # modified
    # driver.find_element_by_xpath(
        # "//a[@href='" + prof_url + "']").click()  # personalized link to go to each professor's website
    driver.get(prof_url)    # added
    page_num = 0
    driver.implicitly_wait(30)
    try:
#         driver.execute_script("")
        load_more_button = driver.find_element_by_css_selector(".Buttons__Button-sc-19xdot-1")  # PaginationButton__StyledPaginationButton-txi1dr-1 eaZArN
        #driver.execute_script("arguments[0].scrollIntoView();", load_more_button)     # added
        driver.implicitly_wait(10)
    except Exception as e:
        print(e)
        load_more_button=None
    #load_more_button=None
    while load_more_button:
        try:
            #driver.execute_script("arguments[0].scrollIntoView();", load_more_button)  # added
            load_more_button.click()
            page_num += 1
            print(page_num)
            time.sleep(2)
        except Exception as e:
            print("Uh oh!!")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # finds all of the elements in the html page that have a "ul" tag and id of "ratingsList"
    unorderedlist = soup.find_all("ul", id="ratingsList")

    # creates a list of all of the individual reviews and saves it under "reviews"
    reviews = unorderedlist[0].find_all("li") if(len(unorderedlist)==1) else []
    inner_count = 0
    outer_count = 0
    #print(reviews)
    for ind in reviews:
        # checks to make sure that all of the list items in the list are actually reviews
        divs = ind.find_all("div", class_="Rating__StyledRating-sc-1rhvpxz-1 jcIQzP")
        outer_count += 1
        if len(divs) != 0:
            inner_count += 1
            course_name = ind.find_all("div", class_="RatingHeader__StyledClass-sc-1dlkqw1-2 gxDIt")
            if len(course_name) != 0:
                course = course_name[0].get_text()
                # print(course)

            time_stamp = ind.find_all("div",
                                      class_="TimeStamp__StyledTimeStamp-sc-9q2r30-0 bXQmMr RatingHeader__RatingTimeStamp-sc-1dlkqw1-3 BlaCV")
            if len(time_stamp) != 0:
                times = time_stamp[0].get_text()
                # print(times)

            review = ind.find_all("div", class_="Comments__StyledComments-dzzyvm-0 gRjWel")
            if len(review) != 0:
                reviews = review[0].get_text()
                # print(reviews)

            textbook_req = ind.find_all("div", class_="MetaItem__StyledMetaItem-y0ixml-0 LXClX")
            credit, attendance, take_again, textbook, online, grade = "", "", "", "", "", ""
            if len(textbook_req) != 0:
                for info in textbook_req:
                    text = info.get_text()
                    str1 = "For Credit: "
                    str2 = "Attendance: "
                    str3 = "Would Take Again: "
                    str4 = "Textbook: "
                    str5 = "Online Class: "
                    str6 = "Grade: "
                    
                    if text.find(str1) != -1:
                        start_index = text.find(str1) + len(str1)
                        credit = text[start_index:]
                        #print(credit)
                    if text.find(str2) != -1:
                        start_index = text.find(str2) + len(str2)
                        attendance = text[start_index:]
                       # print("**************inside",attendance)
                    if text.find(str3) != -1:
                        start_index = text.find(str3) + len(str3)
                        take_again = text[start_index:]
                        #print(take_again)
                    if text.find(str4) != -1:
                        start_index = text.find(str4) + len(str4)
                        textbook = text[start_index:]
                        # print(textbook)
                    if text.find(str5) != -1:
                        start_index = text.find(str5) + len(str5)
                        online = text[start_index:]
                        # print(online)
                    if text.find(str6) != -1:
                        start_index = text.find(str6) + len(str6)
                        grade = text[start_index:]
                        # print(grade)
           # print("**************outside",attendance)
            quality,difficulty="",""
            qualdiff = ind.find_all("div", class_="RatingValues__RatingContainer-sc-6dc747-1 DObVa")
            for elem in qualdiff:
                sub_qualdiff = elem.find_all("div")
                if len(sub_qualdiff) >= 2:
                    if sub_qualdiff[0].get_text().startswith("Quality"):
                        quality = sub_qualdiff[0].get_text()[7:]
                    if sub_qualdiff[0].get_text().startswith("Difficulty"):
                        difficulty = sub_qualdiff[0].get_text()[10:]

            helpful, unhelpful = "", ""
            help_ful = ind.find_all("div", class_="RatingFooter__ButtonWrapper-ciwspm-1 cwcCIQ")
            for elem in help_ful:
                sub_help_ful = elem.find_all("div")
                if len(sub_help_ful) >= 2:
                    helpful = sub_help_ful[0].get_text()
                    unhelpful = sub_help_ful[1].get_text()
                    # print(helpful, unhelpful)

            # gets the various tags for each review
            tags = []
            teacher_tags = ind.find_all("div", class_="RatingTags__StyledTags-sc-1boeqx2-0 eLpnFv")
            for elem in teacher_tags:
                all_tags = elem.find_all("span", class_="Tag-bs9vf4-0 hHOVKF")
                for tag in all_tags:
                    tags.append(tag.get_text())
                    # print(elem.get_text())
            department=""
            dept = soup.find_all("div", class_="NameTitle__Title-dowf0z-1 iLYGwn")
            if len(dept) != 0:
                department = dept[0].get_text()
                start_index = department.find("the ") + len("the ")
                end_index = department.find("department")
                dept_name = department[start_index:end_index]
                # print(dept_name)
            
            prof_name = soup.find_all("div", class_="NameTitle__Name-dowf0z-0 cfjPUG")
            if len(prof_name) != 0:
                prof = prof_name[0].get_text()
            #       print(prof)

            values = [prof, dept_name, times, course, quality, difficulty, reviews, credit, attendance, grade,
                      take_again, textbook, online, helpful, unhelpful]
            for elem in tags:
                values.append(elem)
            for data in values:
                worksheet.write(row, column, str(data))
                #print(row,str(data))
                #print("Printing val")
                column += 1
            row += 1
            column = 0
    #workbook.close()
    print(inner_count)
#     except :
#         print("Error")

def main():
    university_name = "California State University Long Beach"

    # Enters in the university name into the search bar
    search_bar = driver.find_element_by_css_selector(".Search__DebouncedSearchInput-sc-10lefvq-1")
    search_bar.send_keys(university_name)
    search_bar.send_keys(Keys.ENTER)

    print("Entered in the university name")

    # Clicks the appropriate university name out of the list that comes up
    uni_name_link = driver.find_element_by_xpath("//a[@aria-label='Link to school page for " + university_name + "']")
    # scrolls to the top of the window so that the element is visible
    driver.execute_script("window.scrollTo(0,0);", uni_name_link)
    driver.implicitly_wait(10)
    print("before click")
    uni_name_link.click()
    print("Clicked university name")

    # Clicks on the button to view all of the professors in the university
    #view_all_btn = driver.find_element_by_xpath("//a[@class='button']")
    view_all_btn = driver.find_element_by_id("schoolNav")
    #driver.implicitly_wait(20)
    view_all_btn.click()
    print("Entered view all page")

    # Clicks on the "Load More" button to load all of the professors so that they can be analyzed later
    page_num = 0
    #load_more_btn1 = driver.find_element_by_css_selector(".content")
    load_more_btn1 = driver.find_element_by_css_selector(".content")
    driver.execute_script("arguments[0].scrollIntoView();", load_more_btn1)     # added
    load_more_btn1.click() # added
    #load_more_btn1=None
    while load_more_btn1:
        try:
            driver.execute_script("arguments[0].scrollIntoView();", load_more_btn1)
            load_more_btn1.click()
            page_num += 1
            print(page_num)
            time.sleep(1)
        except Exception as e:
            print(e)
            print("All professors have been loaded")
            break
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # finds the div with the list of professors (based on id)
    prof_section = soup.find_all("li", id=lambda value: value and value.startswith("my-professor"))
    #print(prof_section)
    corr_index = 0
    prof_list=[]
    for i in range(0, len(prof_section)):
        #print(prof_section[i])
        prof_url = prof_section[i].find_all("a",href=True)[0]["href"]
        prof_list.append(prof_url)
        #actual_list = inner[0].find_all("li")
        #name = actual_list[0].get('id')
        #if (name is not None) and (name.find("my-professor-") != -1):
        #    corr_index = i
        #    break

    #prof_section_inner = prof_section[corr_index].find_all("ul")1
    #prof_list = prof_section_inner[0].find_all("li")

    # Goes through the entire list of professors to gather review data for each one
    count = 1
    for i in range(0,len(prof_list)):   # can change limit in the range part; for entire thing, write: for i in range(0, len(prof_list): 
        #list_id = prof_list[i].get('id')
        #id_num = list_id[(list_id.find("my-professor-") + len("my-professor-")):]
        print(prof_list[i])
        review_ratings(prof_list[i])
        print(count)
        count += 1
    #workbook.save("professorRatings.xlsx")
    workbook.close()
    


if __name__ == '__main__':
    main()

Entered in the university name
before click
Clicked university name
Entered view all page
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
Message: element not interactable
  (Session info: chrome=89.0.4389.82